Unitary designs are used when there is a need for random sampling or randomizations. More precisely, the Haar measure provides a way to sample uniformly from the group of unitary matrices, but generating truly random unitaries with respect to the Haar measure can be computationally expensive. Unitary designs offer an efficient way to approximate this randomness. A unitary design is a collection of unitary matrices that, when averaged together, behaves like an integral over the Haar measure.
The 2-desings specifically are used for operation benchmarking on qubits. If an operation is perfectly act on a qubit state as $\Lambda (|0\rangle \langle0|)$ it could be expressed as $V|0\rangle \langle0|V^{\dagger}$. And the fidelity measures their overlapping, i.e. $F = \langle 0|V^{\dagger} \Lambda(|0\rangle \langle0|) V |0\rangle $ 

$\bar F(\Lambda, V) = \int \, d\mu(U) \langle 0 | U^\dagger V^\dagger \Lambda (U |0\rangle \langle 0 | U^\dagger) V U |0 \rangle$


The above expressions defines the average fidelity over the unitaries by using Haar measure. This is exactly the case where the unitary t-designs come handy. The equation can be expressed in an exact form as 

$ \int \, d\mu(U) \langle 0 | U^\dagger V^\dagger \Lambda (U |0\rangle \langle 0 | U^\dagger) V U |0 \rangle = \frac{1}{K} \sum\limits_{j=1}^{K} \langle 0 | U_j^\dagger V^\dagger \Lambda (U_j |0\rangle \langle 0 | U_j^\dagger) V U_j |0 \rangle$


Here $K$ is the cardinality of the set of unitaires that form a 2-design.

It can be shown that the Clifford goup is a unitary-3 design so it is a 2 and 1 design, too. For the 1-qubit Clifford group we have the Hadamard ($H$) gate and the phase ($S$) gate as the generator of the Clifford group. Altogether the group consists of 24 elements. So over this 24 elements we need to perform the summation written above.

In [1]:
single_qubit_cliffords = [
 '',
 'H', 'S',
 'HS', 'SH', 'SS',
 'HSH', 'HSS', 'SHS', 'SSH', 'SSS',
 'HSHS', 'HSSH', 'HSSS', 'SHSS', 'SSHS',
 'HSHSS', 'HSSHS', 'SHSSH', 'SHSSS', 'SSHSS',
 'HSHSSH', 'HSHSSS', 'HSSHSS'
]

Here we will check if using the Clifford group for the evaluation of the average fidelity gives the same result as the definition, i.e. integrating over the Haar measure by using a number of randomly sampled unitaries.

In [3]:
import pennylane as qml
import numpy as np

# Scipy for sampling Haar-random unitaries
from scipy.stats import unitary_group

# set the random seed
np.random.seed(42)

# Use the mixed state simulator
dev = qml.device("default.mixed", wires=1)

In [ ]:
# We set up the experiment by defining an operation which we hit by some noise.
# This will make the fidelity diverge from unity for certain.

def ideal_experiment():
    qml.SX(wires=0)
    return qml.state()


def noisy_operations(damp_factor, depo_factor, flip_prob):
    qml.AmplitudeDamping(damp_factor, wires=0)
    qml.DepolarizingChannel(depo_factor, wires=0)
    qml.BitFlip(flip_prob, wires=0)


@qml.qfunc_transform
def apply_noise(tape, damp_factor, depo_factor, flip_prob):
    # Apply the original operations
    for op in tape.operations:
        qml.apply(op)

    # Apply the noisy sequence
    noisy_operations(damp_factor, depo_factor, flip_prob)

    # Apply the original measurements
    for m in tape.measurements:
        qml.apply(m)

# TBC ...